In [2]:
pip install fastapi uvicorn streamlit tensorflow opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
pip install python-multipart

In [ ]:
import os
import cv2
import urllib.request
import zipfile
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from fastapi import FastAPI, File, UploadFile
import uvicorn
import streamlit as st
import requests
from PIL import Image

# Define constants
DATASET_URL = 'https://dicom5c.blob.core.windows.net/public/Data.zip'
DATASET_DIR = 'brain_mri_dataset'
IMG_SIZE = (256, 256)  # Image size for input to the models

# Step 1: Download and extract the dataset
def download_and_extract_dataset(url, extract_to):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    dataset_zip_path = os.path.join(extract_to, 'Data.zip')
    print(f"Downloading dataset from {url}...")
    urllib.request.urlretrieve(url, dataset_zip_path)
    print("Download complete!")

    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Dataset extracted to {extract_to}.")
    os.remove(dataset_zip_path)

# Step 2: Preprocess images (CLAHE and normalization)
def apply_clahe(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(img)

def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = apply_clahe(img)
    img = cv2.resize(img, IMG_SIZE)  # Resize to (256, 256)
    img = img / 255.0  # Normalize to [0, 1]
    return img

# Step 3: Define U-Net++ (Nested U-Net)
def unet_plus_plus(input_size=(256, 256, 1)):
    inputs = layers.Input(input_size)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)

    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = layers.Concatenate()([up1, conv2])
    up1 = layers.Conv2D(128, 3, activation='relu', padding='same')(up1)
    up1 = layers.Conv2D(128, 3, activation='relu', padding='same')(up1)

    up2 = layers.UpSampling2D(size=(2, 2))(up1)
    up2 = layers.Concatenate()([up2, conv1])
    up2 = layers.Conv2D(64, 3, activation='relu', padding='same')(up2)
    up2 = layers.Conv2D(64, 3, activation='relu', padding='same')(up2)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(up2)
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 4: Load data and preprocess
def load_data():
    images = []
    masks = []
    images_path = os.path.join(DATASET_DIR, 'images')
    masks_path = os.path.join(DATASET_DIR, 'masks')

    for img_file in os.listdir(images_path):
        img_path = os.path.join(images_path, img_file)
        mask_path = os.path.join(masks_path, img_file)  # Assuming same file names for masks
        if os.path.exists(mask_path):
            img = preprocess_image(img_path)
            mask = preprocess_image(mask_path)
            images.append(img)
            masks.append(mask)

    images = np.expand_dims(np.array(images), axis=-1)
    masks = np.expand_dims(np.array(masks), axis=-1)
    return images, masks

# Step 5: Model training and evaluation
def train_model():
    images, masks = load_data()
    X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

    model = unet_plus_plus(input_size=(256, 256, 1))
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=16)
    model.save('nested_unet.h5')
    print("Model training complete and saved as nested_unet.h5")

# Step 6: FastAPI backend for prediction
app = FastAPI()

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    img = cv2.imdecode(np.frombuffer(contents, np.uint8), cv2.IMREAD_UNCHANGED)
    img = preprocess_image(img)
    img = np.expand_dims(img, axis=[0, -1])  # Shape: (1, 256, 256, 1)

    # Load model and predict
    model = models.load_model('nested_unet.h5')
    prediction = model.predict(img)
    return {"prediction": prediction.tolist()}

if __name__ == "__main__":
    # Step 7: Download dataset, train model, and run FastAPI server
    download_and_extract_dataset(DATASET_URL, DATASET_DIR)
    train_model()
    uvicorn.run(app, host="0.0.0.0", port=8000)